In [ ]:
import os
import warnings
import numpy as np
import seaborn as sns
from imutils import paths
from google.colab import drive    # Only required if working on Google Colab
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, multilabel_confusion_matrix

from tensorflow import one_hot
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Input, Dense, Conv2D, Dropout, Flatten, AveragePooling2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator

sns.set()
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [ ]:
drive.mount('/content/drive')    # Only required if working on Google Colab

## UDFs

In [ ]:
# Helper function to calculate ROC-AUC score using
# one-vs-all method for each class and the average score
def multiclass_roc_auc_score(actual_class, pred_class, average="macro"):
    unique_class = list(map(np.unique, actual_class))
    roc_auc_dict = dict()
    auc_sum = 0

    for per_class in unique_class: 
        other_class = [x for x in unique_class if x != per_class]

        new_actual_class = [0 if x in other_class else 1 for x in actual_class]
        new_pred_class = [0 if x in other_class else 1 for x in pred_class]

        roc_auc = roc_auc_score(new_actual_class, new_pred_class, average=average).round(4) * 100
        roc_auc_dict[per_class[0]] = roc_auc
        auc_sum += roc_auc
      
    roc_auc_dict['avg'] = auc_sum / len(unique_class)

    return roc_auc_dict

## Data Cleaning & Pre-processing

In [ ]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
train_path = "/content/drive/MyDrive/Colab Data/Face Mask Dataset/train data"
trainImagePaths = list(paths.list_images(train_path))

test_path = "/content/drive/MyDrive/Colab Data/Face Mask Dataset/test data"
testImagePaths = list(paths.list_images(test_path))

In [ ]:
train_data = list()
test_data = list()
labels = list()

# loop over the train image paths
for imagePath in trainImagePaths:

	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the input image (224x224) as grayscale
	image = load_img(imagePath, target_size=(224, 224), color_mode="grayscale")
 
  # convert the image to numpy array
	image = img_to_array(image)
 
  # scale pixel values between -1 and 1
	image = preprocess_input(image)

	# update the train data and labels lists
	train_data.append(image)
	labels.append(label)
 
# loop over the test image paths
for imagePath in testImagePaths:

	# load the input image (224x224) as grayscale
	image = load_img(imagePath, target_size=(224, 224), color_mode="grayscale")
 
  # convert the image to numpy array
	image = img_to_array(image)
 
  # scale pixel values between -1 and 1
	image = preprocess_input(image)

	# update the data
	test_data.append(image)

In [ ]:
# convert the data and labels to NumPy arrays
train = np.array(train_data, dtype="float32")
test = np.array(test_data, dtype="float32")
labels = np.array(labels)

In [ ]:
# save the numpy arrays to use them later
np.save('/content/drive/MyDrive/Colab Data/Face Mask Dataset/numpy arrays/train data', train)
np.save('/content/drive/MyDrive/Colab Data/Face Mask Dataset/numpy arrays/test data', test)
np.save('/content/drive/MyDrive/Colab Data/Face Mask Dataset/numpy arrays/labels', labels)

In [ ]:
# load the numpy arrays
train = np.load('/content/drive/MyDrive/Colab Data/Face Mask Dataset/numpy arrays/train data (grayscale).npy')
test = np.load('/content/drive/MyDrive/Colab Data/Face Mask Dataset/numpy arrays/test data (grayscale).npy')
labels = np.load('/content/drive/MyDrive/Colab Data/Face Mask Dataset/numpy arrays/labels.npy')

In [ ]:
# perform one-hot encoding on the labels
lb = LabelEncoder()
labels_enc = lb.fit_transform(labels)
labels_enc = np.array(one_hot(labels_enc, depth=3))

In [ ]:
# training and evaluation split at 66:33
train_x, eval_x, train_y, eval_y = train_test_split(train, labels_enc, stratify=labels_enc, 
                                                    test_size=0.33, random_state=1)

In [ ]:
train_x.shape, eval_x.shape, train_y.shape, eval_y.shape

In [ ]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2,
                         height_shift_range=0.2, shear_range=0.15, horizontal_flip=True)

## Modelling

#### Training

In [ ]:
# define the input shape for the model
input_tensor = Input(shape=(224, 224, 1))

# add a convolution layer to convert the inputs to 
# 224x224x3 shape for the MobileNet model
input_model= Conv2D(filters=3, kernel_size=3, 
                    padding='same', name="input_conv")(input_tensor)

# load the MobileNetV2 network, ensuring the head FC layer sets are left off
base_model = MobileNetV2(weights="imagenet", 
                         include_top=False,
                         input_tensor=Input(shape=(224, 224, 3)))(input_model)

# construct the head of the model that will be placed on top of the the base model
head_model = AveragePooling2D(pool_size=(7, 7))(base_model)
head_model = Flatten(name="flatten")(head_model)
head_model = Dense(128, activation="relu")(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(3, activation="softmax")(head_model)

# place the head FC model on top of the base model 
# which will become the actual model that we will train
model = Model(inputs=input_tensor, outputs=head_model)

In [ ]:
# view the model architecture
model.summary()

In [ ]:
# loop over all layers in the input & base model and freeze them so they will
# not be updated during the first training process
for layer in model.layers:
	# selecting layer by name
    if layer.name in ["input_conv", "mobilenetv2_1.00_224"]:
        layer.trainable = False

In [ ]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
INIT_LR = 1e-4
EPOCHS = 100
BS = 100

# compile the model
optimizer = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics="accuracy")

In [ ]:
# train the head of the network
H = model.fit(aug.flow(train_x, train_y, batch_size=BS),
              steps_per_epoch=len(train_x)//BS,
              epochs=EPOCHS,
              validation_data=(eval_x, eval_y),
            #   callbacks=EarlyStopping(monitor="val_accuracy", 
            #                           patience=4,
            #                           restore_best_weights=True),
              workers=512,
              use_multiprocessing=True)

#### Evaluation

In [ ]:
train_pred_y = np.argmax(model.predict(train_x, batch_size=BS), axis=1)
pred_y = np.argmax(model.predict(eval_x, batch_size=BS), axis=1)

In [ ]:
print(classification_report(eval_y.argmax(axis=1), pred_y, target_names=lb.classes_))

In [ ]:
print("Training Accuracy:", accuracy_score(train_y.argmax(axis=1), train_pred_y).round(4) * 100)
print("Testing Accuracy:", accuracy_score(eval_y.argmax(axis=1), pred_y).round(4) * 100)

In [ ]:
print("Training ROC-AUC:", multiclass_roc_auc_score(train_y.argmax(axis=1), train_pred_y))
print("Testing ROC-AUC:", multiclass_roc_auc_score(eval_y.argmax(axis=1), pred_y))

In [ ]:
# save the model, weights and environment variables
model.save("/content/drive/MyDrive/Colab Data/Face Mask Dataset/model")

In [ ]:
multilabel_confusion_matrix(eval_y.argmax(axis=1), pred_y)

#### Testing

In [ ]:
# model = load_model("/content/drive/MyDrive/Colab Data/Face Mask Dataset/model")

In [ ]:
face = np.expand_dims(test[561], axis=0)
array_to_img(test[561])

In [ ]:
np.argmax(model.predict(face), axis=1)[0]

#### Plotting

In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.figure(dpi=140)
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center")
plt.show()